In [2]:
import re
import urllib.parse
import urllib.request
import json

In [3]:
url = 'https://www.quebec512.info/en/Carte/Element.ashx'
action = 'PosteFrontalier'

def make_get_request(url: str, **kwargs):
    params = urllib.parse.urlencode(kwargs)
    url = f"{url}?{params}"
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read().decode())

    return data

In [4]:
border_ids = make_get_request(url=url, action=action, lang='en', zoom=8, xMin=-78.25, yMin=42.19, xMax=-68.11, yMax=49.41)

In [ ]:
border_ids 

[{'lng': -70.39708,
  'lat': 45.80577,
  'ico': 'b_PosteFrontalier.png',
  'id': '2',
  'info': 'Armstrong / Jackman'},
 {'lng': -73.08488,
  'lat': 45.01553,
  'ico': 'b_PosteFrontalier.png',
  'id': '20',
  'info': 'St-Armand/Philipsburg / Highgate Springs'},
 {'lng': -73.45264,
  'lat': 45.00887,
  'ico': 'b_PosteFrontalier.png',
  'id': '21',
  'info': 'Saint-Bernard-de-Lacolle / Champlain'},
 {'lng': -71.79345,
  'lat': 45.0107,
  'ico': 'b_PosteFrontalier.png',
  'id': '24',
  'info': 'Stanhope / Norton'},
 {'lng': -72.0883,
  'lat': 45.00591,
  'ico': 'b_PosteFrontalier.png',
  'id': '25',
  'info': 'Stanstead / Derby Line'}]

In [5]:
def extract_selected_elements(data: list, keys: list):
    result = []
    for item in data:
        selected_elements = {key: item[key] for key in keys if key in item}
        result.append(selected_elements)
    return result

In [14]:
url_info = 'https://www.quebec511.info/en/Carte/Fenetres/FenetrePosteFrontalier.aspx'

def get_info_html(url: str, **kwargs):
    params = urllib.parse.urlencode(kwargs)
    url = f"{url}?{params}"
    with urllib.request.urlopen(url) as response:
        html = response.read().decode('utf-8')

    return html

In [9]:
info_html = get_info_html(url=url_info, id=21)

In [15]:
def extract_waittime(html):
    re_start = re.compile('tdAttenteVoyageurAmericain')
    to_can = re_start.split(info_html)[1].split('</td>')[0].strip('\"')
    re_start = re.compile('tdAttenteVoyageurCanadien')
    to_us = re_start.split(info_html)[1].split('</td>')[0].strip('\"')
    return {'Travellers to CAN:': to_can, 'Travellers to US': to_us}

In [11]:
extract_waittime(info_html)

{'Travellers to CAN:': '>No delay', 'Travellers to US': '>No Delay'}

In [ ]:
def add_waittime(data: list):
    url_info = 'https://www.quebec511.info/en/Carte/Fenetres/FenetrePosteFrontalier.aspx'
    result = []
    for d in data:
        id = d['id']
        info_html = get_info_html(url=url_info, id=id)
        wt_dict = extract_waittime(info_html)
        result.append(d|wt_dict)
    return result


In [16]:
add_waittime(border_ids)

[{'lng': -70.39708,
  'lat': 45.80577,
  'ico': 'b_PosteFrontalier.png',
  'id': '2',
  'info': 'Armstrong / Jackman',
  'Travellers to CAN:': '>No delay',
  'Travellers to US': '>No Delay'},
 {'lng': -73.08488,
  'lat': 45.01553,
  'ico': 'b_PosteFrontalier.png',
  'id': '20',
  'info': 'St-Armand/Philipsburg / Highgate Springs',
  'Travellers to CAN:': '>No delay',
  'Travellers to US': '>No Delay'},
 {'lng': -73.45264,
  'lat': 45.00887,
  'ico': 'b_PosteFrontalier.png',
  'id': '21',
  'info': 'Saint-Bernard-de-Lacolle / Champlain',
  'Travellers to CAN:': '>No delay',
  'Travellers to US': '>No Delay'},
 {'lng': -71.79345,
  'lat': 45.0107,
  'ico': 'b_PosteFrontalier.png',
  'id': '24',
  'info': 'Stanhope / Norton',
  'Travellers to CAN:': '>No delay',
  'Travellers to US': '>No Delay'},
 {'lng': -72.0883,
  'lat': 45.00591,
  'ico': 'b_PosteFrontalier.png',
  'id': '25',
  'info': 'Stanstead / Derby Line',
  'Travellers to CAN:': '>No delay',
  'Travellers to US': '>No Delay'}]

In [ ]:
def compile_data():
    border_ids = make_get_request(url=url, action=action, lang='en', zoom=8, xmin=-78.25, ymin=42.19, xmax=-68.11, ymax=49.41)
    return add_waittime(border_ids)